# W209 Final Project: Meeting Efficiency and Effectiveness
## Exploratory Data Analysis - Hypothesis Development
### Author: Naga Chandrasekaran
### Project Concept:
Collaboration has a significant value by which it enables organizations to achieve their goals. In any organization, all jobs are dependent on cross-functional teams of individuals coming together to address the challenge they face collectively. Besides, collaboration spurs innovation. Such collaboration happens in various settings (examples include one-to-one discussion, brainstorming sessions, team discussions, ad-hoc meetings, structured updates) pursued by different organizations and individuals. All these forums are focused on bringing people together to share their ideas and solve problems. In the post-COVID-19 pandemic effect, even an informal talk is a zoom meeting. There is an increasing number of meetings that occupy our calendars. While these meetings are essential, an overwhelming number of them can be energy-draining and prevent individuals from doing their job and utilizing their time more efficiently. This can also affect their work-life balance and impact morale. Meetings are a means to an end, helping individuals achieve assigned goals. One has to wonder if the sessions are efficient and enable us to achieve our goals. Besides, managers and individuals prioritize how they spend their time on topics to get their valuable time slots. This project aims to provide visualizations that help individuals and managers achieve the goal of understanding how their and their team's time is spent during a workday and how it can be streamlined to better their chances of success.

Data used in this project was provided by instructor Fereshteh Amini. We are using this data to perform mock analysis and create visualizations, develop hypothesis, and understand if we are able to answer th hypothesis. This report presents initial exploratory data analysis and hypothesis.

In [1]:
# Import standard libraries needed for the analysis
import altair as alt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

pd.__version__

'1.1.5'

## **Exploratory Data Analysis**

Load the file provided by Fereshteh Amini and look at the data structure and information in the file. 

In [3]:
df = pd.read_csv('FAminDummyMeetingData.csv')
print("There are %i number of rows and %i number of columns" %(df.shape[0], df.shape[1]))
df.head()

There are 288043 number of rows and 39 number of columns


,MeetingId,StartDate,StartTimeUTC,EndDate,EndTimeUTC,Total_redundant_hours,Total_meeting_cost,Redundant_attendees,Invitees,Emails_sent_during_meetings,Attendees_with_conflicting_meetings,Attendees_multitasking,Attendees,Attendee_meeting_hours,Organizer_Domain,Organizer_FunctionType,Organizer_LevelDesignation,Organizer_Layer,Organizer_Region,Organizer_Organization,Organizer_HireDate,Organizer_attainment,Organizer_SupervisorIndicator,Organizer_Number_of_directs,Organizer_TimeZone,Organizer_HourlyRate,Organizer_IsInternal,Organizer_PersonId,IsCancelled,DurationHours,IsRecurring,Subject,TotalAccept,TotalNoResponse,TotalDecline,TotalNoEmailsDuringMeeting,TotalNoDoubleBooked,TotalNoAttendees,MeetingResources
0,040000008200E00074C5B7101A82E00800000000057C5A...,4/2/2020,21:45:00,4/2/2020,22:45:00,0.0,187.5,0,5,6,4,1,4,2.5,WPAPUBLICDEMO.ONMICROSOFT.COM,Sales,Manager,3,US,Finance-West,4/8/2015,MEDIUM,Mgr,4,Pacific Standard Time,75,True,522DD5C54328959D078B6C70DB2EFEC31EA3D0536F0C2B...,False,1.00,False,hold new demo and every review,1,3,1,6,4,4,CONF17
1,040000008200E00074C5B7101A82E0080000000003A69B...,2/6/2020,00:00:00,2/6/2020,00:30:00,2.0,225.0,0,7,2,3,0,7,3.0,WPAPUBLICDEMO.ONMICROSOFT.COM,Operations,Junior IC,3,US,G&A Central,3/16/2017,NaN,IC,0,Pacific Standard Time,75,True,3C2797E85824D8745E76E99847785FA0A0C7AF2E298ED1...,False,0.50,False,one projects of will,4,3,0,2,3,7,CONF12
2,040000008200E00074C5B7101A82E0080000000005A835...,4/2/2020,21:45:00,4/2/2020,23:00:00,0.0,187.5,0,3,2,3,1,3,2.5,WPAPUBLICDEMO.ONMICROSOFT.COM,Marketing,Support,3,US,G&A Central,10/26/2018,NaN,IC,0,Pacific Standard Time,75,True,C3ABE20D2CC62026C23AD63129D79D93B0EA4452714ED6...,False,1.25,False,the presentation up every functional,1,2,0,2,3,3,CONF1
3,040000008200E00074C5B7101A82E0080000000014138A...,2/6/2020,00:00:00,2/6/2020,01:00:00,3.0,225.0,0,5,3,5,0,5,3.0,WPAPUBLICDEMO.ONMICROSOFT.COM,Operations,Support,2,US,Finance-West,9/25/2016,NaN,IC,0,Pacific Standard Time,75,True,44D26ADEDBF3640313C8473F207D16D5407E6E52CCFE6B...,False,1.00,False,russ and a demo from reporting sales off,1,4,0,3,5,5,CONF16
4,040000008200E00074C5B7101A82E008000000000B7006...,4/2/2020,21:45:00,4/2/2020,23:15:00,7.5,787.5,0,7,6,0,0,7,10.5,WPAPUBLICDEMO.ONMICROSOFT.COM,Engineering,Support,3,US,G&A South,11/26/2016,NaN,IC,0,Pacific Standard Time,75,True,16032331F169120B70F20E1C8C89DF4FA0712B3BFBE546...,False,1.50,False,the every messaging quick,7,0,0,6,0,7,CONF11


In [4]:
for col in df.columns: 
  print(col) 

MeetingId
StartDate
StartTimeUTC
EndDate
EndTimeUTC
Total_redundant_hours
Total_meeting_cost
Redundant_attendees
Invitees
Emails_sent_during_meetings
Attendees_with_conflicting_meetings
Attendees_multitasking
Attendees
Attendee_meeting_hours
Organizer_Domain
Organizer_FunctionType
Organizer_LevelDesignation
Organizer_Layer
Organizer_Region
Organizer_Organization
Organizer_HireDate
Organizer_attainment
Organizer_SupervisorIndicator
Organizer_Number_of_directs
Organizer_TimeZone
Organizer_HourlyRate
Organizer_IsInternal
Organizer_PersonId
IsCancelled
DurationHours
IsRecurring
Subject
TotalAccept
TotalNoResponse
TotalDecline
TotalNoEmailsDuringMeeting
TotalNoDoubleBooked
TotalNoAttendees
MeetingResources


In [5]:
df.describe()

,Total_redundant_hours,Total_meeting_cost,Redundant_attendees,Invitees,Emails_sent_during_meetings,Attendees_with_conflicting_meetings,Attendees_multitasking,Attendees,Attendee_meeting_hours,Organizer_Layer,Organizer_Number_of_directs,Organizer_HourlyRate,DurationHours,TotalAccept,TotalNoResponse,TotalDecline,TotalNoEmailsDuringMeeting,TotalNoDoubleBooked,TotalNoAttendees
count,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000,288043.0,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000,288043.000000
mean,2.548231,334.590990,0.029312,5.566582,2.715560,1.852977,0.243585,5.127596,4.686318,2.607267,1.174068,75.0,1.047390,3.614408,1.513187,0.438987,2.715560,1.852977,5.127596
std,3.939185,385.219935,0.315356,2.265680,4.055285,2.005525,0.556444,2.099572,5.456911,0.565781,2.466112,0.0,1.039936,1.917208,1.992402,0.511810,4.055285,2.005525,2.099572
min,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,75.0,0.250000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,112.500000,0.000000,4.000000,0.000000,0.000000,0.000000,4.000000,1.500000,2.000000,0.000000,75.0,0.500000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,1.250000,206.250000,0.000000,5.000000,1.000000,1.000000,0.000000,5.000000,3.000000,3.000000,0.000000,75.0,0.750000,3.000000,1.000000,0.000000,1.000000,1.000000,5.000000
75%,3.000000,375.000000,0.000000,7.000000,3.000000,3.000000,0.000000,7.000000,5.250000,3.000000,0.000000,75.0,1.000000,5.000000,3.000000,1.000000,3.000000,3.000000,7.000000
max,49.000000,5381.250000,4.000000,15.000000,67.000000,12.000000,6.000000,15.000000,79.000000,3.000000,16.000000,75.0,7.750000,12.000000,13.000000,2.000000,67.000000,12.000000,15.000000


# Initial Data Look

The data file has significant amount of data. Each entry is identified by meeting ID. The file has a total of 288043 rows (meeting entries) and 39 columns (information about each meeting).

The time period for the data in the file is a little over one year. It covers entire calendar year 2020 and some additional data points in 2021. For practical purposes, we can consider that the data is for 13 months.

Each meeting ID provides information about a meeting. Individual meetings have information about their start and end time, cost of meeting, number of attendees, focus of the attendees in the meeting, meeting conflicts, information about the organizer level and organization of meeting organizer, and some additional information.

However, there are also several missing information, which would have been nice to have for our analysis. Information does not provide details into which organizations are attending a meeting, details about individuals who are attending the meeting, and the details behind the meeting. There is a subject column, which has text information and we need to analyze this to see if we can pull any commons. We cannot get information about average number of hours individual team members are attending meetings without making certain assumptions. So, analysis has to be at an organization level.

In the sections below, we will provide data analysis, visualizations, and initial hypothesis development.

## **Meeting Count By Organizational Functional Area**

*   Which organization schedules most meetings?
*   Are all organizations similar or is there a spread?



In [6]:
# Count number of meetings by each organization functional area
df1 = df.copy()
df1["Meeting_Count"] = df1.groupby('Organizer_FunctionType')["MeetingId"].transform('count') 

# Create a new dataframe with these columns
new_df1 = df1[['Organizer_FunctionType', 'Meeting_Count']].copy()

# Drop duplicate values
new_df1 = new_df1.drop_duplicates(subset = ['Organizer_FunctionType'])
new_df1.head(10)

,Organizer_FunctionType,Meeting_Count
0,Sales,34473
1,Operations,42593
2,Marketing,78828
4,Engineering,24900
5,Finance,30991
10,R_and_D,32468
11,G_and_A,30968
17,IT,12822


In [11]:
# Number of meetings by Functional Organization
alt.Chart(new_df1).mark_bar().encode(
    alt.X('Organizer_FunctionType', sort="-y", axis=alt.Axis(title="Functional Organization")),
    alt.Y('Meeting_Count', axis=alt.Axis(title="Number of Meetings")),
).interactive().properties(title='Number of Meetings By Organization Type').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=600,height=300
    )

alt.Chart(...)

The Marketing organization schedules the most number of meetings by meeting count. It is almost 2X compared to its nearest neighbor. Operations, Sales, R&D, and Finance close out the top 5 organizations with meetings. IT has the lowest number of meetings. This is as expected except we did not expect IT organization to be lowest for number of meetings.

In the data, while we see information about whether the organizer is internal, it does not provide information about if the attendees or the scope of the meeting is internal or external. We would expect that many of the marketing and sales meeting are external. We checked if any of the meeting organizer is external and found that 100% of meeting organizers are internal to the organization.

From the data, we cannot answer the network of meetings. For example, if we want to answer if sales and marketing are attending same meetings, we cannot get this information. Likewise, how many collaboration meetings are happening between R&D and engineering, we cannot answer such questions.

## **Meeting Count By Organizer Level in the Organization**

*   What is the level of the individual scheduling meetings?
*   Is there a trend based on the meeting organizer level in the organization?



In [12]:
# Count number of meetings by Organizer level desgnation
df2 = df.copy()
df2["Organizer_Count"] = df2.groupby('Organizer_LevelDesignation')["MeetingId"].transform('count') 

# Create a new dataframe with these columns
new_df2 = df2[['Organizer_LevelDesignation', 'Organizer_Count']].copy()

# Drop duplicate values
new_df2 = new_df2.drop_duplicates(subset = ['Organizer_LevelDesignation'])
new_df2

,Organizer_LevelDesignation,Organizer_Count
0,Manager,39756
1,Junior IC,27872
2,Support,113478
5,Senior IC,82102
7,Director,21263
80,Senior Executive,786
147,Executive,2786


In [13]:
# Number of meetings by Functional Organization
alt.Chart(new_df2).mark_bar().encode(
    alt.X('Organizer_LevelDesignation', sort="-y", axis=alt.Axis(title="Organizer Level")),
    alt.Y('Organizer_Count', axis=alt.Axis(title="Number of Meetings")),
).interactive().properties(title='Number of Meetings By Organizer Type').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=600, height=200
    )

alt.Chart(...)

It is interesting to see that the most meetings are organized by the employees in support category. Depending on how support is defined, this is not surprising. Without the details, we are making an assumption here. If executive administrators and meeting organizers are classified as support group, most of the meetings will be organized by them. Also, as the management level designation of an individual increases, the number of meetings scheduled by them decreases. Let us look at organizer supervisor indicator to see if it sheds any light.

In [15]:
# Count number of meetings by Organizer level desgnation
df_sup = df.copy()
df_sup["Organizer_Sup_Count"] = df_sup.groupby('Organizer_SupervisorIndicator')["MeetingId"].transform('count') 

# Create a new dataframe with these columns
new_dfsup = df_sup[['Organizer_SupervisorIndicator', 'Organizer_Sup_Count']].copy()

# Drop duplicate values
new_dfsup = new_dfsup.drop_duplicates(subset = ['Organizer_SupervisorIndicator'])
new_dfsup

,Organizer_SupervisorIndicator,Organizer_Sup_Count
0,Mgr,39756
1,IC,223452
7,Mgr+,24835


We cannot make major conclusions from the organizer supervisor indicator other than many of the support individuals might be reporting to a manager or manager+ designation.

## **Meeting Cost**

*   Is the average cost of meeting higher as organizer level is higher?



In [16]:
# Group by organizer level and get average meeting cost
df3 = df.copy()
df3["Meeting_Cost"] = df3.groupby('Organizer_LevelDesignation')["Total_meeting_cost"].transform('mean') 

# Create a new dataframe with these columns
new_df3 = df3[['Organizer_LevelDesignation', 'Meeting_Cost']].copy()

# Drop duplicate values
new_df3 = new_df3.drop_duplicates(subset = ['Organizer_LevelDesignation'])

In [17]:
# Average cost of meeting by the organizer level in organization
alt.Chart(new_df3).mark_bar().encode(
    alt.X('Organizer_LevelDesignation', sort="-y", axis=alt.Axis(title="Organizer Level")),
    alt.Y('Meeting_Cost', axis=alt.Axis(title="Average Cost of Meeting")),
).interactive().properties(title='Average Meeting Cost By Organizer Type').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=600, height=200
    )

alt.Chart(...)

The details on how cost of meeting is measured is not provided. If we assume that cost of meeting is based on attendees in the meeting (their salary, time spent in the meeting), then we would expect that as the meeting organizer level increases, the average meeting cost will go up because the attendees might be higher level. 

From the data, we see that meeting average cost is highest when the organizer level is manager. Surprisingly the average meeting cost is lower when organizer level is senior executive or Director or executive level. This might indicate that the meeting cost is referencing to something different.

In [18]:
# Plot cost vs duration hours for meeting (Total_Meeting_Cost vs DurationHours)
dfs = df.sample(5000)
dfs["avg_meeting_duration"] = dfs.groupby('Organizer_FunctionType')["DurationHours"].transform('mean')
dfs["avg_meeting_cost"] = dfs.groupby('Organizer_FunctionType')["Total_meeting_cost"].transform('mean')
meet_cost_df = dfs[['Organizer_FunctionType', 'avg_meeting_duration', 'avg_meeting_cost']].copy()

alt.Chart(meet_cost_df).mark_circle(size=100).encode(
    alt.X('avg_meeting_duration', axis=alt.Axis(title="Average Meeting Duration"),
          scale=alt.Scale(domain=(0.5, 1.5))), 
    alt.Y('avg_meeting_cost', axis=alt.Axis(title="Average Meeting Cost"),
          scale = alt.Scale(domain=(200, 450))),
    color=alt.Color('Organizer_FunctionType', legend=alt.Legend(title="Org Function")),
    tooltip=['Organizer_FunctionType', 'avg_meeting_duration', 'avg_meeting_cost']
).interactive().properties(title='Meeting cost vs Duration by Org function').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24)

alt.Chart(...)

We take a sample of data set and group them by organizer function and measure the average meeting duration and average meeting cost. We plot this data of average meeting cost as a function of average meeting duration for different organizer functions. We use color as encoding here and use circle markers. 

From the data, we can conclude that sales meetings are longest and most expensive in the organization. This is followed by Marketing, Operations, G&A, and Finance. Note that sales, Marketing, and operations also had high number of meetings. R&D, IT, and engineering are lower for meeting cost. This trend lines up with number of meeting trends except that marketing had highest number of meetings but sales is highest in terms of average meeting cost and duration.

## **Duration Hours By Organization Function**

As we noted from the earlier graph, we wanted to see average meeting hours by organization type and can clearly see that sales is highest in terms of meeting duration followed by Marketing, operations and other organizations.

In [19]:
# Average hours of meetings by each organization functional area
df4 = df.copy()
df4["Meeting_Hours"] = df4.groupby('Organizer_FunctionType')["DurationHours"].transform('mean') 

# Create a new dataframe with these columns
new_df4 = df4[['Organizer_FunctionType', 'Meeting_Hours']].copy()

# Drop duplicate values
new_df4 = new_df4.drop_duplicates(subset = ['Organizer_FunctionType'])

In [20]:
# Average hours of meetings by each organization functional area
alt.Chart(new_df4).mark_bar().encode(
    alt.X('Organizer_FunctionType', sort="-y", axis=alt.Axis(title="Functional Organization")),
    alt.Y('Meeting_Hours', axis=alt.Axis(title="Average Heeting Hours")),
).interactive().properties(title='Average Meeting Hours By Organization Type').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=600,height=300
    )

alt.Chart(...)

# **Meeting count by number of invitees**

*   How many attendees are invited to the meetings?
*   Is there a trend between meeting hours and number of attendees in a meeting?
*   Does type of organization play a role in this trend?




In [ ]:
# Take a sample set of the data to reduce calculation time
# Meeting count by number of invitees
df_time = df.sample(5000)
alt.Chart(df_time).mark_bar().encode(
    alt.X('binned_invitees:O', axis=alt.Axis(title="Binned Invitees")),
    alt.Y('count()', sort='-x',axis=alt.Axis(title="Number of Meetings")),
).transform_bin('binned_invitees', field='Invitees').interactive().properties(
    title='Meeting count by number of invitees').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=800, height=200
    )

Output hidden; open in https://colab.research.google.com to view.

We create a binned histogram for number or meetings as a function of number of meeting invitees. We use a sample of 5000 meetings for this analysis. We can see that the maximum number of meetings, over 60% have between 4 to 6 attendees.

# **Meeting count by meeting duration**

*   What is the duration of meetings?


In [21]:
# Take a sample set of the data to reduce calculation time
# Meeting Duration Hours
df_duration = df.sample(5000)
bars = alt.Chart(df_duration).mark_bar().encode(
    alt.X("DurationHours", bin=alt.Bin(extent=[0, 10], step=1)),
    y='count()',
)

text = bars.mark_text(
    align='center',
    baseline='bottom',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='count():O'
)

(bars + text).interactive().properties(
    title='Meeting count by Duration').configure_axis(
        labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
            width=800, height=200
    )

Output hidden; open in https://colab.research.google.com to view.

In [27]:
# Plot Meeting count vs duration by organizer function type
dfc = df.sample(5000)
dfc["avg_meeting_duration"] = dfc.groupby('Organizer_FunctionType')["DurationHours"].transform('mean')
dfc["avg_meeting_count"] = dfc.groupby('Organizer_FunctionType')["MeetingId"].transform('count')
meet_duration_df = dfc[['Organizer_FunctionType', 'avg_meeting_duration', 'avg_meeting_count']].copy()

alt.Chart(meet_duration_df).mark_circle(size=100).encode(
    alt.X('avg_meeting_duration', axis=alt.Axis(title="Average Meeting Duration"),
          scale=alt.Scale(domain=(0.5, 1.5))), 
    alt.Y('avg_meeting_count', axis=alt.Axis(title="Meeting Count"),
          scale = alt.Scale(domain=(0, 2000))),
    color=alt.Color('Organizer_FunctionType', legend=alt.Legend(title="Org Function")),
    tooltip=['Organizer_FunctionType', 'avg_meeting_duration', 'avg_meeting_count']
).interactive().properties(title='Meeting count vs Duration by Org function').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24)

alt.Chart(...)

For the sample of 5000 meetings, we see that over ~50% of meetings are in the 1 hour or less duration. Remaining meetings are greater than one hour but less than 1.5 hours. We can conclude that the meeting durations are pretty consistent and there are not many long meetings.

We have completed some initial high level EDA of the data and have gained some good initial understanding of the overall data set, what is represented in the data, and what can be done with it. As we continue to analyze the data, let us form some initial hypothesis and see if the data can support or dispute our initial hypothesis.

## **Hypothesis 1: Over 50% of the meetings are not effective**

*   With the number of meetings, cost of meetings, and level of individuals involved in meetings, we want to understand if the meetings are efficient and effective. We have already seen that the duration of the meetings are primarily in the one hour or less time. This can be considered as one measure of effectiveness as well. What other data features can be used to measure effectiveness?

> Effectiveness Measures: Effectiveness is measured by following vectors:
*   Emails sent during meetings
*   Attendees multitasking

*   We assume that if more emails are sent during the meetings and higher number of attendees are multitasking, then the individuals are not focused about meeting content and outcome. Individuals are distracted and the meetings are not adding value to the individuals or the individuals are not adding value to the meetings thus reducing the meeting effectiveness.








In [28]:
df["Emails_sent_during_meetings"].value_counts()

0     86448
1     63231
2     41913
3     26751
4     17669
      ...  
63        1
65        1
58        1
54        1
67        1
Name: Emails_sent_during_meetings, Length: 64, dtype: int64

In [29]:
a = {'Emails': ["0", "1", "2", "3", "4", 
                "Above 4"], 'Meeting Percentage': [30, 22, 15, 9, 6, 18]}
Ecount_df = pd.DataFrame(data=a)

alt.Chart(Ecount_df).mark_bar().encode(
    alt.X('Emails', sort="-y", axis=alt.Axis(title="Number of emails during meeting")),
    alt.Y('Meeting Percentage', axis=alt.Axis(title="Percentage of Meetings")),
).interactive().properties(title='Percentage of Meetings By Number of Emails').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=600,height=300
    )

alt.Chart(...)

Out of the total 288,043 meetings, approximately 86,448 meetings have zero emails sent during the meetings. This represents roughly 30% of total meetings. Over 50% of meetings have atleast 1-4 emails sent during the meeting. An additional 18% of meetings have >4 emails sent during the meetings. Over 70% of the meetings have at least 1 email sent during the meeting.

In [30]:
# Take a sample set of the data to reduce calculation time
# Look at binning of number of meetings by number of emails sent
df7  = df.sample(5000)

bars = alt.Chart(df7).mark_bar().encode(
    alt.X("Emails_sent_during_meetings", bin=alt.Bin(extent=[1, 60], step=5)),
    y='count()',
)

text = bars.mark_text(
    align='center',
    baseline='bottom',
    dx=3
).encode(
    text='count():O'
)

(bars + text).interactive().properties(
    title='Meeting count by number of emails during meetings').configure_axis(
        labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
            width=800, height=200
    )

Output hidden; open in https://colab.research.google.com to view.

Out of the 5000 meetings sampled here, we can clearly see again that ~80% of the meetings have between 1-5 emails sent during the meetings.

In addition, we can see that the data supports around 18% of meetings having >4 emails sent during the meeting. We see 563 meetings (~10%) with 5-10 emails during the meeting. It is interesting to se meetings with >20 emails sent during the meeting.

Next, let us review the average number of emails sent during meetings by the organization type. For this analysis we are excluding meetings with zero emails sent during the meetings.

In [31]:
# Average emails sent per meeting by organizer_function type (excluding zero email meetings)
df8 = df[df['Emails_sent_during_meetings'] 
                    > 0].groupby('Organizer_FunctionType')['Emails_sent_during_meetings'].mean()
Email_average = pd.DataFrame(df8)
Email_average

,Emails_sent_during_meetings
Organizer_FunctionType,
Engineering,3.388509
Finance,3.341262
G_and_A,3.303140
IT,2.959815
Marketing,4.589110
Operations,3.753986
R_and_D,3.349142
Sales,4.549259


In [32]:
d = {'Organization': ["Engineering", "Finance", "G&A", "IT", "Marketing", "Operations", 
                      "R&D", "Sales"], 'Average_Emails': [3.39, 3.34, 3.3, 2.96,
                                                          4.59, 3.75, 3.35, 4.55]}
Email_df = pd.DataFrame(data=d)

alt.Chart(Email_df).mark_bar().encode(
    alt.X('Organization', sort="-y", axis=alt.Axis(title="Functional Organization")),
    alt.Y('Average_Emails', axis=alt.Axis(title="Average Emails (excluding zeros)")),
).interactive().properties(title='Average Emails During Meetings By Organization Type').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=600,height=300
    )

alt.Chart(...)

When we consider meetings with at least one email sent during the meeting, and then look for average number of emails sent during the meeting (meetings with zero emails excluded), we can conclude that Marketing and Sales meetings rank highest in terms of average number of emails sent during the meetings. This is followed by Operations. Finance, G&A, Engineering, and R&D have equal number of emails sent. IT has the lowest number of emails sent during the meeting.

We also noted earlier that Marketing has the highest number of meetings followed by Sales and Operations. Engineering was lower in the number of meetings list but it has higher average number of emails sent during meetings. But in general, we can conclude that organizations with higher number of meetings also have on average higher number of emails sent during the meetings.

Next, let us consider based on meeting organizer level designation if there are emails being sent.

In [33]:
# Average emails sent per meeting by organizer_function type (excluding zero email meetings)
df9 = df[df['Emails_sent_during_meetings'] 
                    > 0].groupby('Organizer_LevelDesignation')['Emails_sent_during_meetings'].mean()
Email_level = pd.DataFrame(df9)
Email_level

,Emails_sent_during_meetings
Organizer_LevelDesignation,
Director,3.469877
Executive,3.440275
Junior IC,3.871377
Manager,4.044511
Senior Executive,2.585308
Senior IC,3.929198
Support,3.876787


In [34]:
c = {'Level Designation': ["Director", "Executive", "Junior IC", "Manager",
                           "Sr. Executive", "Senior IC", "Support"], 
     'Average Emails Sent': [3.47, 3.44, 3.87, 4.04, 2.59, 3.93, 3.88]}
Level_df = pd.DataFrame(data=c)

alt.Chart(Level_df).mark_bar().encode(
    alt.X('Level Designation', sort="-y", axis=alt.Axis(title="Organizer Level Designation")),
    alt.Y('Average Emails Sent', axis=alt.Axis(title="Average Emails Sent")),
).interactive().properties(title='Average Emails Sent by Meeting Organizer Designation').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=20).properties(
        width=600,height=300
    )

alt.Chart(...)

Fewer emails are sent when the meeting organizer designation is senior executive. As meeting organizer designation lowers there are more average emails sent during the meeting.

As next step, let us start looking at number of attendees who are multitasking during the meetings to see if this variable can give us insight into meeting effectiveness.

In [35]:
df["Attendees_multitasking"].value_counts()

0    232359
1     44170
2      9094
3      1939
4       421
5        56
6         4
Name: Attendees_multitasking, dtype: int64

In [36]:
b = {'Number of Attendees Multitasking': ["0", "1", "2", ">3"], 
     'Percentage Meetings': [80.67, 15.3, 3.2, 0.8]}
Mcount_df = pd.DataFrame(data=b)

alt.Chart(Mcount_df).mark_bar().encode(
    alt.X('Number of Attendees Multitasking', sort="-y", axis=alt.Axis(title="Number of Attendees Multitasking")),
    alt.Y('Percentage Meetings', axis=alt.Axis(title="Percentage of Meetings")),
).interactive().properties(title='Percentage of Meetings With Team member multitasking').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=20).properties(
        width=600,height=300
    )

alt.Chart(...)

In [37]:
df10 = df.copy()
df10["multitask_Count"] = df10.groupby('Organizer_LevelDesignation')["Attendees_multitasking"].transform('sum') 

# Create a new dataframe with these columns
new_df10 = df10[['Organizer_LevelDesignation', 'multitask_Count']].copy()

# Drop duplicate values
new_df10 = new_df10.drop_duplicates(subset = ['Organizer_LevelDesignation'])

alt.Chart(new_df10).mark_bar().encode(
    alt.X('Organizer_LevelDesignation', sort="-y", axis=alt.Axis(title="Organizer Level")),
    alt.Y('multitask_Count', axis=alt.Axis(title="Number of Multitaskers")),
).interactive().properties(title='Number of Multitaskers in Meetings By Organizer Type').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=600, height=200
    )

alt.Chart(...)

We can see that a majority of meetings have zero attendees multitasking. This is a good indication. While >70% meetings have team members sending emails, the data also indicates that >80% of meetings do not have team members multitasking. Why is this discrepancy in the data? Is sending emails not considered as multitasking? It depends on how we define multitasking. So, for our purposes, we can consider sending emails as multitasking and use the analysis of emails sent during the meetings as a measure of effectiveness. Based on this we can come to a conclusion for our hypothesis.

# **Hypothesis 1 Conclusion**

Based on our analysis, we can conclude that emails are being sent in more than 50% of the meetings. 

On average, between 2.8 to 4 emails are being sent in meetings. Marketing, Sales, and Operations driven meetings have higher average number of emails being sent. Organizations with higher number of meetings also experience higher number of average emails sent during the meetings. This can be indicative of meeting effectiveness. We can also extrapolate and assume that with more meetings, team members have less time to take care of their individual responsibility (like sending emails) and are having to do this during a meeting.

Least number of emails are sent when the meeting is organized by a senior executive. This is expected.

Attendees multitasking data is not useful to drive any conclusions. Over 80% of meetings have zero attendees multitasking but the definition of multitasking is not clear.

Based on these initial data analysis, we can conclude that >50% of meetings are not effective and have some level of distraction during the meetings thus supporting our initial hypothesis.

# **Hypothesis 2: Outside of workhour meetings are impacting work-life balance**

We want to review if meetings are happening during weekends and outside the 8-5 work hours.

*   Review the density of meetings by work day and timing
*   Review the density of meetings by organization type
*   Review density of meetings by organizer level designation

## **Meeting Work Hours**

What are typical hours when meetings are happening?

In [51]:
# Create functions to convert start time and day dataset
# Copy original data frame into a new dataframe
time_df = df.copy()

# Create a function to convert date to day of week
def ConvertDate2Day(date):
    day, month, year = (int(x) for x in date.split('/'))
    if month > 12:
        month = 12
    ans = datetime.date(year, month, day)
    return ans.strftime("%A")

# Create a function to convert UTC time to Hour w/ AM/PM designation
def ConvertUTC2Hour(time):
    # Perform 5 hour to convert UTC to Eastern Time Zone
    hour = int(time[0:2]) - 5
    if hour < 0:
        hour = 23+hour
    if hour < 12:
        hour = str(hour) +" AM"
    elif hour == 12:
        hour = str(hour) +" PM"
    else:
        hour = str(hour-12) + " PM"
    return hour

# Add calculated fields to data frame
time_df['StartDay'] = time_df['StartDate'].apply(ConvertDate2Day)
time_df['StartHour'] = time_df['StartTimeUTC'].apply(ConvertUTC2Hour)

In [52]:
# Plot bar graph that shows meeting start time by time of day
# Reduce total data set and look at a sample of data
samp_time_df = time_df.sample(5000)

bars = alt.Chart(samp_time_df).mark_bar().encode(
    alt.X("StartHour"),
    y='count()')

text = bars.mark_text(
    align='center',
    baseline='bottom',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='count():O'
)

(bars + text).interactive().properties(
    title='Meeting count by Start Hour').configure_axis(
        labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
            width=800, height=200
    )

Output hidden; open in https://colab.research.google.com to view.

From the bar graph above, we can see that number of meetings are uniform from 10am to 5 pm. Peak meetings seem to happen around the 3 pm point and taper on either side of the 3 p.m. This organization starts relatively late and the meetings begin at 9am and pick up speed at 10am. Meeting load decreases after 5 pm.

Let us look at the day of the week when the meetings are happening.

## **Meeting Work Days**

What are typical days of week for meeting work load?

In [53]:
# Plot bar graph of data that shows meetings by day of the week
bars = alt.Chart(samp_time_df).mark_bar().encode(
    alt.X("StartDay"), #bin=alt.Bin(extent=[0, 10], step=1)),
    y='count()',
)

text = bars.mark_text(
    align='center',
    baseline='bottom',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='count():O'
)

(bars + text).interactive().properties(
    title='Meeting count by Start Day').configure_axis(
        labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
            width=800, height=200
    )

Output hidden; open in https://colab.research.google.com to view.

From the bar graph above, we can see that the meetings pick up during the second half of the week. Mondays and Tuesdays are relatively slow. Meetings are peak on Wednesday. Thursday and Friday are also high. Surprising to see meetings on Saturday and Sunday as well. This is a U.S. organization but we see meetings on Saturday and Sunday.

Are these meetings in Saturday and Sunday happening only during the pandemic period? This organization seems to mostly have meetings only during working hours (9-5 p.m.) but shows meetings during weekends as well.

## **Create heatmap of meetings by work hour and day of week**

In [54]:
# Plot heatmap of the dataset
plot_df = time_df.groupby(['StartDay', 'StartHour']).size().reset_index(name='Meeting Count')

alt.Chart(plot_df).mark_rect().encode(
    x=alt.X('StartHour', title='Meeting Start Hour',
            sort=['1 AM', '2 AM', '3 AM', '4 AM', '5 AM', '6 AM', '7 AM', '8 AM', '9 AM', '10 AM', '11 AM', '12 PM',
                  '1 PM', '2 PM', '3 PM', '4 PM', '5 PM', '6 PM', '7 PM', '8 PM', '9 PM', '10 PM', '11 PM', '12 AM']),
    y=alt.Y('StartDay', title='Meeting Day of Week', 
            sort=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']),
    color='Meeting Count'
).properties(width=600, height=300)

alt.Chart(...)

From the dataset above, we can see that most of the meetings are happening during workhours. Some meetings happen post 5 p.m. But for a majority of the meetings, they happen between 9am to 6pm. Also, the meetings are mostly during weekdays but we see some meetings on Saturdays and Sundays.

We can conclude that the work time is not unreasonable but why do we have meetings during weekend? Are these meetings due to certain specific groups?

## **Meetings by organizer function and time/day of week**

We created an interactive graph that plots the heat map of meetings by day of week and time of day by organizer function of meeting count. We can click on the organizer function where it shows the meeting count and simultaneously changes the heat map for meetings. This is an interactive graph that can give more insight into which organizations are facing challenges with work-life balance indicated by more meetings during off work time and day of week.

In [55]:
# Include Organizer Department in data frame
org_plot_df = time_df.groupby(['StartDay', 'StartHour', 
                               'Organizer_FunctionType']).size().reset_index(
                                   name='Meeting Count')

# Build selection points (used in bar graph)
pts = alt.selection(type="single", encodings=['x'])

# Build Heat Map of Meeting Start Hour vs Meeting Day of Week 
# Filter on selection from bar chart Organizer Department
rect = alt.Chart(org_plot_df).mark_rect().encode(
    x=alt.X('StartHour', title='Meeting Start Hour',
            sort=['1 AM', '2 AM', '3 AM', '4 AM', '5 AM', '6 AM', '7 AM', '8 AM', '9 AM', '10 AM', '11 AM', '12 PM',
                  '1 PM', '2 PM', '3 PM', '4 PM', '5 PM', '6 PM', '7 PM', '8 PM', '9 PM', '10 PM', '11 PM', '12 AM']),
    y=alt.Y('StartDay', title='Meeting Day of Week', 
            sort=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']),
    # When all orgs selected show summary meeting colors
    color='sum(Meeting Count)'
).properties(
    width=300,
    height=200
).transform_filter(
    pts
)

# Build bar chart that shows total meetings by department
# Enable selection for filter from the bar chart
bar = alt.Chart(org_plot_df).mark_bar().encode(
    x='Organizer_FunctionType:N',
    y='sum(Meeting Count)',
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).properties(
    width=250,
    height=200
).add_selection(pts)

# Horizontal stacking of the plots
alt.hconcat(
    rect,
    bar
).resolve_legend(
    color="independent",
    size="independent"
)

alt.HConcatChart(...)

We plot the same graph now with circles as markers inside the heat map that shows the meeting count or number of meetings.

In [56]:
# Build selection points (used in bar graph)
pts = alt.selection(type="single", encodings=['x'])

# Build Heat Map of Meeting Start Hour vs Meeting Day of Week 
# Filter on selection from bar chart Organizer Department 
rect = alt.Chart(org_plot_df).mark_rect().encode(
    x=alt.X('StartHour', title='Meeting Start Hour',
            sort=['1 AM', '2 AM', '3 AM', '4 AM', '5 AM', '6 AM', '7 AM', '8 AM', '9 AM', '10 AM', '11 AM', '12 PM',
                  '1 PM', '2 PM', '3 PM', '4 PM', '5 PM', '6 PM', '7 PM', '8 PM', '9 PM', '10 PM', '11 PM', '12 AM']),
    y=alt.Y('StartDay', title='Meeting Day of Week', 
            sort=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']),
    # Show Sum of meetings for all orgs
    color='sum(Meeting Count)'
).properties(
    width=250,
    height=200
)

# Add circles to encode meeting count by department on top of 
# Filter on selection from bar chart Organizer Department
circ = rect.mark_point().encode(
    alt.ColorValue('grey'),
    alt.Size('sum(Meeting Count)',
        legend=alt.Legend(title='Meeting Count in Selection')
    )
).transform_filter(
    pts
)

# Build bar chart that shows total meetings by department
# Enable selection for filter from the bar chart
bar = alt.Chart(org_plot_df).mark_bar().encode(
    x='Organizer_FunctionType:N',
    y='sum(Meeting Count)',
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).properties(
    width=250,
    height=250
).add_selection(pts)

# Horizontal Stack and Plot with Legend
alt.hconcat(
    rect + circ,
    bar
).resolve_legend(
    color="independent",
    size="independent"
)

alt.HConcatChart(...)

# **Hypothesis 2 Conclusions:**

From the data analysis, we can conclude that meetings happen during proper work time. However, across the organization meetings also happen during weekends. While the number of meetings is lower, it is not negligible. So, it appears that there is a work-life balance concern.

We need to normalize the meeting count as percentage of meetings in weekends for different organizations and see if certain organizations have a higher percentage of meetings during weekends. We now have different types of graphs and visualizations to be able to add this feature and have more clarity in proving the hypothesis.

# **Hypothesis 3: Number of meetings have increased during the pandemic period (Mar 2020 to Nov 2020)**

*   Review meeting count over time
*   Compare meeting count during the period of interest and outside the period of interest
*   Create graphics that allows us to compare by organization




In [57]:
# Count number of meetings by Date
df5 = df.copy()
df5["Meeting_Count"] = df5.groupby('StartDate')["MeetingId"].transform('count') 

# Create a new dataframe with these columns
new_df5 = df5[['StartDate', 'Meeting_Count']].copy()

# Drop duplicate values
new_df5 = new_df5.drop_duplicates(subset = ['StartDate'])

In [58]:
alt.Chart(new_df5).mark_line().encode(
    x='StartDate:T',
    y='Meeting_Count:Q'
).interactive().properties(title='Number of Meetings By Start Date').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=800, height=200
    )

alt.Chart(...)

From the graph, we see an interesting trend. Data for meetings is available from early 2020 to current 2021 timeline. Between April 2020 to November 2020, the data is more continuous. However, the data for number of meetings prior to March 2020 and post November 2020 have lot of spikes. It is not clear if this is a result of data issues or if it follows weekend patterns. In other words, was workload too high and meetings happening even during weekends in the time period of Mar 2020 to November 2020?
Why do we see this unique meeting pattern between prior to Mar 2020 and post Nov 2020 (or) is the period between Mar 2020 to Nov 2020 different?

In [61]:
# Count number of meetings by Date
meet_date_df = df.copy()
meet_date_df["Date_Count"] = meet_date_df.groupby(['StartDate', 'Organizer_FunctionType'])["MeetingId"].transform('count')  
meet_date_df["Date_Duration"] = meet_date_df.groupby(['StartDate', 'Organizer_FunctionType'])["DurationHours"].transform('sum')

# Simplify dataframe to select columns
meet_date_df = meet_date_df[['StartDate','Organizer_FunctionType', 'Date_Count', 'Date_Duration']].copy()

# Drop duplicate values
meet_date_df = meet_date_df.drop_duplicates(subset = ['StartDate', 'Organizer_FunctionType'])


In [62]:
# Redo the above graph using points, color by function, and duration by point size
alt.Chart(meet_date_df).mark_point().encode(
  alt.X('StartDate:T', title='Date'),
  alt.Y('sum(Date_Count)',title='Meetings per Day'),
  size=alt.Size('Date_Duration:Q',scale=alt.Scale(range=[5,  100])),
  color = 'Organizer_FunctionType:N'
).interactive().properties(title='Number of Meetings By Start Date').configure_axis(
    labelFontSize=15,titleFontSize=15,).configure_title(fontSize=24).properties(
        width=800, height=200
    )

alt.Chart(...)

Based on the above two charts, we can clearly see that the marketing organization has the most number of meetings per day and the duration is also high. We saw similar trends and data previously. IT is lowest.

In [63]:
color = alt.Color('Organizer_FunctionType:N')
# Create two selections
# - a brush that is active on the top panel
# - a multi-click that is active on the bottom panel
brush = alt.selection_interval(encodings=['x'])
click = alt.selection_multi(encodings=['color'])

#--------------------------------------------------------
# 1. Top panel is scatter plot of meeting count vs time
points = alt.Chart(meet_date_df).mark_point().encode(
  alt.X('StartDate:T', title='Date'),
  alt.Y('sum(Date_Count)',title='Meetings per Day'),
  color=alt.condition(brush, color, alt.value('lightgray')),
  size=alt.Size('Date_Duration:Q',scale=alt.Scale(range=[5,  100]))
).properties(
    width=550,
    height=300
).add_selection(
    brush
).transform_filter(
    click
)
    
#--------------------------------------------------------
# 2. Bottom panel is a bar chart of organizer function
bars = alt.Chart(meet_date_df).mark_bar().encode(
    x=alt.X('sum(Date_Count)', title = 'Meeting Count'),
    y=alt.Y('Organizer_FunctionType:N', title = "Meetings per Department"),
    color=alt.condition(click, color, alt.value('lightgray')),
).transform_filter(
    brush
).properties(
    width=550,
).add_selection(
    click
)
#--------------------------------------------------------
#3. Build Compound Plot
alt.vconcat(
    points,
    bars,
    title="Meeting Count and Cumulative Duration by Date"
)

alt.VConcatChart(...)

We now have different time trend graphs and different styles of graphs for us to be able to visaulize and analyze whether the meeting load is worse during pandemic. Only time period prior to pandemic is prior to March 2020. We will have to filter the data and visualize pre and post pandemic to see if meeting loads have increased during the pandemic period. Post Nov 2020, the meeting trend also looks different but this could be holiday season.

At this point, we cannot conclusively say if hypothesis 3 is supported or not. Based on the initial look we would conclude that hypothesis 3 is not supported and we would reject the null hypothesis. Need more study in this area.